# Import libraries

In [ ]:
import os
import pandas as pd
from meteostat import Point, Daily
import requests
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import plotly.express as px
from jinja2 import Template
from dotenv import load_dotenv, dotenv_values

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

%matplotlib inline

load_dotenv()

# Parameters

In [ ]:
mytz = 'Europe/Luxembourg'
lat = 49.619871
lon = 6.172835
alt = 269.5
OWM_API_KEY = os.getenv('OWM_API_KEY')
rapidapi_key = os.getenv('RAPIDAPI_KEY')

# Import metadata

In [ ]:
md = pd.read_csv('metadata.csv')

# Import Atmotube data

In [ ]:
df = pd.read_csv('data.csv', parse_dates=['Date'], date_format='%Y-%m-%d %H:%M:%S')
df['Date'] = df['Date'].dt.tz_localize(mytz)
df.rename(columns={'PM1, ug/m³': 'PM₁ in μg/m³',
                   'PM2.5, ug/m³': 'PM₂.₅ in μg/m³',
                   'PM10, ug/m³': 'PM₁₀ in μg/m³',
                   'VOC, ppm': 'VOC in ppm',
                   'Temperature, ˚C': 'Temperature in ˚C',
                   'Humidity, %': 'Humidity in %',
                   'Pressure, hPa': 'Pressure in hPa'}, inplace=True)

In [ ]:
df.info()

In [ ]:
df = df[['Date',
         'AQS',
         'VOC in ppm',
         'PM₁ in μg/m³', 
         'PM₂.₅ in μg/m³', 
         'PM₁₀ in μg/m³',
         'Temperature in ˚C',
         'Humidity in %',
         'Pressure in hPa',
         'Latitude',
         'Longitude']]

In [ ]:
df.shape

In [ ]:
start_dt = df['Date'].min()
print(f'Start datetime in local time zone is {start_dt}')

end_dt = df['Date'].max()
print(f'End datetime in local time zone is {end_dt}')
print('\n')

start_dt_unix_utc = (pd.Timestamp.tz_convert(start_dt, tz='UTC') - pd.Timestamp("1970-01-01", tz='UTC')) // pd.Timedelta('1s')
print(f'Start datetime in UNIX time, UTC time zone is {start_dt_unix_utc}')

end_dt_unix_utc = (pd.Timestamp.tz_convert(end_dt, tz='UTC') - pd.Timestamp("1970-01-01", tz='UTC')) // pd.Timedelta('1s')
print(f'End datetime in UNIX time, UTC time zone is {end_dt_unix_utc}')
print('\n')

duration = end_dt - start_dt

def days_hours_minutes(td):
    days = td.days
    hours = td.seconds//3600
    minutes = (td.seconds//60)%60
    return f'{days} days, {hours} hours, {minutes} minutes'

print(days_hours_minutes(duration))

# Import OpenWeather air pollution data

In [ ]:
url = f'http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start_dt_unix_utc}&end={end_dt_unix_utc}&appid={OWM_API_KEY}'
response = requests.get(url)
hist_air_data = response.json()

In [ ]:
ow_air = pd.json_normalize(hist_air_data, 'list')
ow_air.rename(columns={'dt': 'Date',
                       'main.aqi': 'AQI', # Air Quality Index
                       'components.co': 'CO in μg/m³', # Carbon monoxide in μg/m³
                       'components.no': 'NO in μg/m³', # Nitrogen monoxide in μg/m³
                       'components.no2': 'NO₂ in μg/m³', # Nitrogen dioxide in μg/m³
                       'components.o3': 'O₃ in μg/m³', # Ozone in μg/m³
                       'components.so2': 'SO₂ in μg/m³', # Sulphur dioxide in μg/m³
                       'components.pm2_5': 'PM₂.₅ in μg/m³', # PM2.5 in μg/m³
                       'components.pm10': 'PM₁₀ in μg/m³', # PM10 in μg/m³
                       'components.nh3': 'NH₃ in μg/m³'}, # Ammonia in μg/m³
              inplace=True)

ow_air['Date'] = pd.to_datetime(ow_air['Date'], unit='s').dt.tz_localize(mytz)

start_dt_ow = ow_air['Date'].min()
print(f'Start datetime in local time zone in OpenWeather data is {start_dt_ow}')

end_dt_ow = ow_air['Date'].max()
print(f'End datetime in local time zone in OpenWeather data is {end_dt_ow}')

# Import Meteostat wind data

In [ ]:
home = Point(lat=lat, lon=lon, alt=alt)
station = home.get_stations()
station = station.loc[station['score'] == station['score'].max()].index.to_list()[0]

url = "https://meteostat.p.rapidapi.com/stations/hourly"

start_dt_meteostat = end_dt - datetime.timedelta(days = 30)

querystring = {"station": station,
               "start": start_dt_meteostat.strftime('%Y-%m-%d'),
               "end": end_dt.strftime('%Y-%m-%d'),
               "tz": mytz,
               "model": "true",
               "freq": "h",
               "units": "metric"}

headers = {
	"x-rapidapi-key": rapidapi_key,
	"x-rapidapi-host": "meteostat.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

hist_wind_data = response.json()

hist_wind_data = pd.json_normalize(hist_wind_data, 'data')

hist_wind_data.drop(columns=['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'pres', 'tsun', 'coco'], inplace=True)

hist_wind_data.rename(columns={'time': 'Date', 
                               'wdir': 'Wind (from) direction in degrees', 
                               'wspd': 'Average wind speed in km/h',
                               'wpgt': 'Wind peak gust in km/h'}, inplace=True)

hist_wind_data['Date'] = pd.to_datetime(hist_wind_data['Date']).dt.tz_localize(mytz)

# Clean Atmotube data

In [ ]:
# Check for duplicate rows
sum(df.duplicated())

In [ ]:
# Check if Date has duplicates
sum(df['Date'].duplicated())

In [ ]:
# Drop duplicates in Date
df['Date'].drop_duplicates(keep='first', inplace=True)

# Create time series
dt_series = pd.Series(pd.date_range(start=start_dt, end=end_dt, freq='min', tz=mytz), name='Date')
df2 = pd.merge(left=dt_series, right=df, how='left', on='Date')

# Sort by Date
df2.sort_values('Date', inplace=True)

# Exclude calibration period
df3 = df2.loc[df2['Date'] >= '2024-07-18 19:07:00']

# Exclude some periods
df3.loc[(df3['Date'] >= '2024-07-19 21:15:00') & (df3['Date'] <= '2024-07-19 22:33:00'), ['VOC in ppm', 'AQS', 'PM₁ in μg/m³', 'PM₂.₅ in μg/m³', 'PM₁₀ in μg/m³']] = None

# Specify coordinates
df3 = df3.drop(columns=['Latitude', 'Longitude'])
df3['Latitude'] = lat
df3['Longitude'] = lon

# Join Atmotube data with OpenWeather data
df4 = pd.merge(left=df3, right=ow_air.drop(columns=['PM₂.₅ in μg/m³', 'PM₁₀ in μg/m³']), how='left', on='Date')

# Replace invalid values
df4.loc[df4['PM₁ in μg/m³'] > 100, 'PM₁ in μg/m³'] = 100
df4.loc[df4['PM₂.₅ in μg/m³'] > 170, 'PM₂.₅ in μg/m³'] = 170
df4.loc[df4['PM₁₀ in μg/m³'] > 250, 'PM₁₀ in μg/m³'] = 250
df4.loc[df4['NO₂ in μg/m³'] == -9999.0, 'NO₂ in μg/m³'] = None

# Join with Meteostat data
df5 = pd.merge(left=df4, right=hist_wind_data, how='left', on='Date')

# Sort by Date
df5.sort_values('Date', inplace=True)

# Replace missing values with the last known value
df6 = df5.ffill(axis=0)

In [ ]:
df6.head()

In [ ]:
df6.info()

## Exceptions

In [ ]:
def CountExceptions(df_in, col):
    labels = md.loc[md['Metric'] == col].sort_values('Sort order')['Qualitative name'].to_list()
    bins = md.loc[md['Metric'] == col]['max'].to_list()
    bins.append(0)
    bins = sorted(bins)
    # print(bins)
    # print(labels)
    df_in[f'{col} group'], bins = pd.cut(x=df_in[col], bins=bins, labels=labels, retbins=True, include_lowest=True)
    df_out = df_in.groupby(f'{col} group', observed=True).agg(['size'])[col]
    df_out = pd.merge(pd.DataFrame(labels, columns=[col]), df_out, left_on=col, right_on=df_out.index, how='left', sort=False)
    df_out = pd.merge(md.loc[md['Metric'] == col], df_out, left_on='Qualitative name', right_on=col, how='left', sort=False)
    df_out = df_out.drop(columns=[col, 'Metric', 'Sort order'])
    df_out['size'] = df_out['size'].fillna(0).astype(int)
    df_out['pct'] = df_out['size'] / len(df_in)
    if col == 'AQI':
        df_out[col] = df_out['min'].astype(int).astype(str)
    elif col in ['AQS', 'PM₁ in μg/m³', 'PM₂.₅ in μg/m³', 'PM₁₀ in μg/m³', 'CO in μg/m³', 'NO₂ in μg/m³', 'O₃ in μg/m³', 'SO₂ in μg/m³']:
        df_out[col] = df_out['min'].astype(int).astype(str) + '-' + df_out['max'].astype(int).astype(str)
    else:
        df_out[col] = df_out['min'].astype(str) + '-' + df_out['max'].astype(str)
    df_out = df_out[[col, 'Qualitative name', 'Notes', 'size', 'pct']]
    df_out.rename(columns={'size': 'Count of measurements', 'pct': 'Percent of measurements'}, inplace=True)
    return df_out
    
aqs_exc = CountExceptions(df_in=df6, col='AQS')
aqi_exc = CountExceptions(df_in=df6, col='AQI')
voc_exc = CountExceptions(df_in=df6, col='VOC in ppm')
pm1_exc = CountExceptions(df_in=df6, col='PM₁ in μg/m³')
pm2_exc = CountExceptions(df_in=df6, col='PM₂.₅ in μg/m³')
pm10_exc = CountExceptions(df_in=df6, col='PM₁₀ in μg/m³')
co_exc = CountExceptions(df_in=df6, col='CO in μg/m³')
no2_exc = CountExceptions(df_in=df6, col='NO₂ in μg/m³')
o3_exc = CountExceptions(df_in=df6, col='O₃ in μg/m³')
so2_exc = CountExceptions(df_in=df6, col='SO₂ in μg/m³')

# Create plots

Line plots

In [ ]:
a = 0.4

analysis_cols = ['AQS', 'VOC in ppm', 'PM₁ in μg/m³', 'PM₂.₅ in μg/m³', 'PM₁₀ in μg/m³', 'Temperature in ˚C', 'Humidity in %', 'Pressure in hPa', # Atmotube data
                 'AQI', 'CO in μg/m³', 'NO in μg/m³', 'NO₂ in μg/m³', 'O₃ in μg/m³', 'SO₂ in μg/m³', 'NH₃ in μg/m³', # OpenWeather data
                 'Wind (from) direction in degrees', 'Average wind speed in km/h', 'Wind peak gust in km/h'] # Meteostat data

for col in analysis_cols:

    if col == 'AQS':
        fig_aqs = px.line(df6, x='Date', y=col, range_y=[0, 100], width=900, height=480)
        fig_aqs.update_traces(line_color='black', line_width=2, hovertemplate='%{x|%d-%b-%Y %H:%M}<br>Value: %{y}')
        fig_aqs.update_layout(yaxis={'showgrid': False},
                              xaxis={'showgrid': False, 'tickformat': '%d-%b-%Y %H:%M'},
                              title={'text': col, 'x': 0.5, 'xanchor': 'center'},
                              plot_bgcolor='rgba(0,0,0,0)',
                              margin=dict(l=10, r=10, t=30, b=10),
                              shapes=[dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=80, y1=100, fillcolor='lightskyblue', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=60, y1=80,  fillcolor='green', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=40, y1=60,  fillcolor='yellow', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=20, y1=40,  fillcolor='red', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=0, y1=20,  fillcolor='darkred', opacity=a, layer='below', line=dict(width=0))])
        fig_aqs.update_xaxes(title='Date', showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside')
        fig_aqs.update_yaxes(title='Value', showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside')

    if col == 'AQI':
        fig_aqi = px.line(df6, x='Date', y=col, range_y=[0.5, 5.5], width=900, height=480)
        fig_aqi.update_traces(line_color='black', line_width=2, hovertemplate='%{x|%d-%b-%Y %H:%M}<br>Value: %{y}')
        fig_aqi.update_layout(yaxis={'showgrid': False},
                              xaxis={'showgrid': False, 'tickformat': '%d-%b-%Y %H:%M'},
                              title={'text': col, 'x': 0.5, 'xanchor': 'center'},
                              plot_bgcolor='rgba(0,0,0,0)',
                              margin=dict(l=10, r=10, t=30, b=10),
                              shapes=[dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=0.5, y1=1.5, fillcolor='lightskyblue', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=1.5, y1=2.5, fillcolor='green', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=2.5, y1=3.5, fillcolor='yellow', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=3.5, y1=4.5, fillcolor='red', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=4.5, y1=5.5, fillcolor='darkred', opacity=a, layer='below', line=dict(width=0))])
        fig_aqi.update_xaxes(title='Date', showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside')
        fig_aqi.update_yaxes(title='Value', showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside')

    if col == 'VOC in ppm':
        fig_voc = px.line(df6, x='Date', y=col, range_y=[0.0, 7.5], width=900, height=480)
        fig_voc.update_traces(line_color='black', line_width=2, hovertemplate='%{x|%d-%b-%Y %H:%M}<br>Value: %{y}')
        fig_voc.update_layout(yaxis={'showgrid': False},
                              xaxis={'showgrid': False, 'tickformat': '%d-%b-%Y %H:%M'},
                              title={'text': col, 'x': 0.5, 'xanchor': 'center'},
                              plot_bgcolor='rgba(0,0,0,0)',
                              margin=dict(l=10, r=10, t=30, b=10),
                              shapes=[dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=0.0, y1=0.3, fillcolor='lightskyblue', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=0.3, y1=1.0, fillcolor='green', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=1.0, y1=2.5, fillcolor='yellow', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=2.5, y1=5.5, fillcolor='red', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=5.5, y1=7.5, fillcolor='darkred', opacity=a, layer='below', line=dict(width=0))])
        fig_voc.update_xaxes(title='Date', showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside')
        fig_voc.update_yaxes(title='Value', showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside')

    if col == 'PM₁ in μg/m³':
        fig_pm1 = px.line(df6, x='Date', y=col, range_y=[0, 100], width=900, height=480)
        fig_pm1.update_traces(line_color='black', line_width=2, hovertemplate='%{x|%d-%b-%Y %H:%M}<br>Value: %{y}')
        fig_pm1.update_layout(yaxis={'showgrid': False},
                              xaxis={'showgrid': False, 'tickformat': '%d-%b-%Y %H:%M'},
                              title={'text': col, 'x': 0.5, 'xanchor': 'center'},
                              plot_bgcolor='rgba(0,0,0,0)',
                              margin=dict(l=10, r=10, t=30, b=10),
                              shapes=[dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=0,  y1=14, fillcolor='lightskyblue', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=14, y1=34, fillcolor='green', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=34, y1=61, fillcolor='yellow', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=61, y1=95, fillcolor='red', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=95, y1=100, fillcolor='darkred', opacity=a, layer='below', line=dict(width=0))])
        fig_pm1.update_xaxes(title='Date', showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside')
        fig_pm1.update_yaxes(title='Value', showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside')


    if col == 'PM₂.₅ in μg/m³':
        fig_pm2 = px.line(df6, x='Date', y=col, range_y=[0, 170], width=900, height=480)
        fig_pm2.update_traces(line_color='black', line_width=2, hovertemplate='%{x|%d-%b-%Y %H:%M}<br>Value: %{y}')
        fig_pm2.update_layout(yaxis={'showgrid': False},
                              xaxis={'showgrid': False, 'tickformat': '%d-%b-%Y %H:%M'},
                              title={'text': col, 'x': 0.5, 'xanchor': 'center'},
                              plot_bgcolor='rgba(0,0,0,0)',
                              margin=dict(l=10, r=10, t=30, b=10),
                              shapes=[dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=0,  y1=20, fillcolor='lightskyblue', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=20, y1=50, fillcolor='green', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=50, y1=90, fillcolor='yellow', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=90, y1=140, fillcolor='red', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=140, y1=170, fillcolor='darkred', opacity=a, layer='below', line=dict(width=0))])
        fig_pm2.update_xaxes(title='Date', showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside')
        fig_pm2.update_yaxes(title='Value', showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside')

    if col == 'PM₁₀ in μg/m³':
        fig_pm10 = px.line(df6, x='Date', y=col, range_y=[0, 250], width=900, height=480)
        fig_pm10.update_traces(line_color='black', line_width=2, hovertemplate='%{x|%d-%b-%Y %H:%M}<br>Value: %{y}')
        fig_pm10.update_layout(yaxis={'showgrid': False},
                              xaxis={'showgrid': False, 'tickformat': '%d-%b-%Y %H:%M'},
                              title={'text': col, 'x': 0.5, 'xanchor': 'center'},
                              plot_bgcolor='rgba(0,0,0,0)',
                              margin=dict(l=10, r=10, t=30, b=10),
                              shapes=[dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=0,  y1=30, fillcolor='lightskyblue', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=30, y1=75, fillcolor='green', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=75, y1=125, fillcolor='yellow', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=125, y1=200, fillcolor='red', opacity=a, layer='below', line=dict(width=0)),
                                      dict(type='rect', xref='paper', x0=0, x1=1, yref='y', y0=200, y1=250, fillcolor='darkred', opacity=a, layer='below', line=dict(width=0))])
        fig_pm10.update_xaxes(title='Date', showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside')
        fig_pm10.update_yaxes(title='Value', showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside')

    # if col == 'Temperature in ˚C':
    #     plt.grid(color='lightgrey')
    #     plt.text(0.05, 0.95, 'Source: Atmotube PRO', horizontalalignment='left', verticalalignment='top', transform=plt.gca().transAxes)
    #     plt.savefig('temp.png', format='jpg', bbox_inches='tight')
    #     plt.close()
 
    # if col == 'Humidity in %':
    #     plt.grid(color='lightgrey')
    #     plt.text(0.05, 0.95, 'Source: Atmotube PRO', horizontalalignment='left', verticalalignment='top', transform=plt.gca().transAxes)
    #     plt.savefig('humidity.png', format='jpg', bbox_inches='tight')
    #     plt.close()

    # if col == 'Pressure in hPa':
    #     plt.grid(color='lightgrey')
    #     plt.text(0.05, 0.95, 'Source: Atmotube PRO', horizontalalignment='left', verticalalignment='top', transform=plt.gca().transAxes)
    #     plt.savefig('pressure.png', format='jpg', bbox_inches='tight')
    #     plt.close()

    # if col == 'CO in μg/m³':
    #     plt.axhspan(0, 4400, facecolor='lightskyblue', alpha=a)
    #     plt.axhspan(4400, 9400, facecolor='green', alpha=a)
    #     plt.axhspan(9400, 12400, facecolor='yellow', alpha=a)
    #     plt.axhspan(12400, 15400, facecolor='red', alpha=a)
    #     plt.axhspan(15400, 18400, facecolor='darkred', alpha=a)
    #     plt.text(0.05, 0.95, 'Source: OpenWeather', horizontalalignment='left', verticalalignment='top', transform=plt.gca().transAxes)
    #     plt.ylim(0, 18400)
    #     plt.savefig('co.png', format='jpg', bbox_inches='tight')
    #     plt.close()

    # if col == 'NO in μg/m³':
    #     plt.grid(color='lightgrey')
    #     plt.ylim(0, 100)
    #     plt.text(0.05, 0.95, 'Source: OpenWeather', horizontalalignment='left', verticalalignment='top', transform=plt.gca().transAxes)
    #     plt.savefig('no.png', format='jpg', bbox_inches='tight')
    #     plt.close()

    # if col == 'NO₂ in μg/m³':
    #     plt.axhspan(0, 40, facecolor='lightskyblue', alpha=a)
    #     plt.axhspan(40, 70, facecolor='green', alpha=a)
    #     plt.axhspan(70, 150, facecolor='yellow', alpha=a)
    #     plt.axhspan(150, 200, facecolor='red', alpha=a)
    #     plt.axhspan(200, 250, facecolor='darkred', alpha=a)
    #     plt.ylim(0, 250)
    #     plt.text(0.05, 0.95, 'Source: OpenWeather', horizontalalignment='left', verticalalignment='top', transform=plt.gca().transAxes)
    #     plt.savefig('no2.png', format='jpg', bbox_inches='tight')
    #     plt.close()

    # if col == 'NH₃ in μg/m³':
    #     plt.grid(color='lightgrey')
    #     plt.ylim(0, 200)
    #     plt.text(0.05, 0.95, 'Source: OpenWeather', horizontalalignment='left', verticalalignment='top', transform=plt.gca().transAxes)
    #     plt.savefig('nh3.png', format='jpg', bbox_inches='tight')
    #     plt.close()

    # if col == 'SO₂ in μg/m³':
    #     plt.axhspan(0, 20, facecolor='lightskyblue', alpha=a)
    #     plt.axhspan(20, 80, facecolor='green', alpha=a)
    #     plt.axhspan(80, 250, facecolor='yellow', alpha=a)
    #     plt.axhspan(250, 350, facecolor='red', alpha=a)
    #     plt.axhspan(350, 450, facecolor='darkred', alpha=a)
    #     plt.text(0.05, 0.95, 'Source: OpenWeather', horizontalalignment='left', verticalalignment='top', transform=plt.gca().transAxes)
    #     plt.ylim(0, 450)
    #     plt.savefig('so2.png', format='jpg', bbox_inches='tight')
    #     plt.close()

    # if col == 'O₃ in μg/m³':
    #     plt.axhspan(0, 60, facecolor='lightskyblue', alpha=a)
    #     plt.axhspan(60, 100, facecolor='green', alpha=a)
    #     plt.axhspan(100, 140, facecolor='yellow', alpha=a)
    #     plt.axhspan(140, 180, facecolor='red', alpha=a)
    #     plt.axhspan(180, 225, facecolor='darkred', alpha=a)
    #     plt.text(0.05, 0.95, 'Source: OpenWeather', horizontalalignment='left', verticalalignment='top', transform=plt.gca().transAxes)
    #     plt.ylim(0, 225)
    #     plt.savefig('o3.png', format='jpg', bbox_inches='tight')
    #     plt.close()

    # if col == 'Wind (from) direction in degrees':
    #     plt.grid(color='lightgrey')
    #     plt.text(0.05, 0.95, 'Source: Meteostat', horizontalalignment='left', verticalalignment='top', transform=plt.gca().transAxes)
    #     plt.ylim(0, 370)
    #     plt.yticks(range(0, 420, 60))
    #     plt.savefig('wdir.png', format='jpg', bbox_inches='tight')
    #     plt.close()

    # if col == 'Average wind speed in km/h':
    #     plt.grid(color='lightgrey')
    #     plt.text(0.05, 0.95, 'Source: Meteostat', horizontalalignment='left', verticalalignment='top', transform=plt.gca().transAxes)
    #     plt.savefig('wspd.png', format='jpg', bbox_inches='tight')
    #     plt.close()

    # if col == 'Wind peak gust in km/h':
    #     plt.grid(color='lightgrey')
    #     plt.text(0.05, 0.95, 'Source: Meteostat', horizontalalignment='left', verticalalignment='top', transform=plt.gca().transAxes)
    #     plt.savefig('wpgt.png', format='jpg', bbox_inches='tight')
    #     plt.close()

Wind rose

In [ ]:
wr = df6[['Wind (from) direction in degrees', 'Average wind speed in km/h', 'Wind peak gust in km/h']].dropna()

def degToCompass(num):
    val=int((num/22.5)+.5)
    arr=["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]
    return arr[(val % 16)]

wr['Wind (from) direction in compass'] = wr['Wind (from) direction in degrees'].apply(degToCompass)
wr['Wind (from) direction in compass'] = pd.Categorical(wr['Wind (from) direction in compass'], ordered=True, categories=["N","NNE","NE","ENE","E","ESE","SE","SSE","S","SSW","SW","WSW","W","WNW","NW","NNW"])

wr['Wind speed group in km/h'] = pd.cut(wr['Average wind speed in km/h'], bins=[0, 5, 10, 15, 20, 25, 30, 35, float("inf")], include_lowest=True, right=False)
wr['Wind peak gust group in km/h'] = pd.cut(wr['Wind peak gust in km/h'], bins=[0, 5, 10, 15, 20, 25, 30, 35, 40, float("inf")], include_lowest=True, right=False)

wra = wr.groupby(['Wind (from) direction in compass', 'Wind speed group in km/h'], observed=False).agg(['size'])['Average wind speed in km/h'].reset_index()
wra.rename(columns={'size': 'Frequency'}, inplace=True)
wra.sort_values(['Wind speed group in km/h', 'Wind (from) direction in compass'], inplace=True)

fig_wra = px.bar_polar(wra, r="Frequency", theta="Wind (from) direction in compass", color='Wind speed group in km/h', template="gridon", color_discrete_sequence=px.colors.sequential.Blues)

wrg = wr.groupby(['Wind (from) direction in compass', 'Wind peak gust group in km/h'], observed=False).agg(['size'])['Wind peak gust in km/h'].reset_index()
wrg.rename(columns={'size': 'Frequency'}, inplace=True)
wrg.sort_values(['Wind peak gust group in km/h', 'Wind (from) direction in compass'], inplace=True)

fig_wrg = px.bar_polar(wrg, r="Frequency", theta="Wind (from) direction in compass", color='Wind peak gust group in km/h', template="gridon", color_discrete_sequence=px.colors.sequential.Blues)

## Statistical analysis

In [ ]:
descriptive_stats = df6.describe()[analysis_cols].T
descriptive_stats['count'] = descriptive_stats['count'].astype(int)

## Prepare HTML file

In [ ]:
int_formatter = lambda x: f'{x:,}'
pct_formatter = lambda x: f'{x:.0%}'

jinja_data = {'fig_aqi': fig_aqi.to_html(),
              'fig_aqs': fig_aqs.to_html(),
              'fig_co': 'co.png',
              'fig_humidity': 'humidity.png',
              'fig_nh3': 'nh3.png',
              'fig_no': 'no.png',
              'fig_no2': 'no2.png',
              'fig_o3': 'o3.png',
              'fig_pm1': fig_pm1.to_html(),
              'fig_pm10': fig_pm10.to_html(),
              'fig_pm2': fig_pm2.to_html(),
              'fig_pressure': 'pressure.png',
              'fig_so2': 'so2.png',
              'fig_temp': 'temp.png',
              'fig_voc': fig_voc.to_html(),
              'fig_wdir': 'wdir.png',
              'fig_wpgt': 'wpgt.png',
              'fig_wspd': 'wspd.png',
              'aqs_exc': aqs_exc.to_html(index=False, border=0, classes='exception_table_bgyrd', justify='left', formatters={'Count of measurements': int_formatter, 'Percent of measurements': pct_formatter}),
              'aqi_exc': aqi_exc.to_html(index=False, border=0, classes='exception_table_drygb', justify='left', formatters={'Count of measurements': int_formatter, 'Percent of measurements': pct_formatter}, escape=False),
              'voc_exc': voc_exc.to_html(index=False, border=0, classes='exception_table_drygb', justify='left', formatters={'Count of measurements': int_formatter, 'Percent of measurements': pct_formatter}),
              'pm1_exc': pm1_exc.to_html(index=False, border=0, classes='exception_table_drygb', justify='left', formatters={'Count of measurements': int_formatter, 'Percent of measurements': pct_formatter}),
              'pm2_exc': pm2_exc.to_html(index=False, border=0, classes='exception_table_drygb', justify='left', formatters={'Count of measurements': int_formatter, 'Percent of measurements': pct_formatter}),
              'pm10_exc': pm10_exc.to_html(index=False, border=0, classes='exception_table_drygb', justify='left', formatters={'Count of measurements': int_formatter, 'Percent of measurements': pct_formatter}),
              'co_exc': co_exc.to_html(index=False, border=0, classes='exception_table_drygb', justify='left', formatters={'Count of measurements': int_formatter, 'Percent of measurements': pct_formatter}),
              'no2_exc': no2_exc.to_html(index=False, border=0, classes='exception_table_drygb', justify='left', formatters={'Count of measurements': int_formatter, 'Percent of measurements': pct_formatter}),
              'o3_exc': o3_exc.to_html(index=False, border=0, classes='exception_table_drygb', justify='left', formatters={'Count of measurements': int_formatter, 'Percent of measurements': pct_formatter}),
              'so2_exc': so2_exc.to_html(index=False, border=0, classes='exception_table_drygb', justify='left', formatters={'Count of measurements': int_formatter, 'Percent of measurements': pct_formatter}),
              'fig_wra': fig_wra.to_html(),
              'fig_wrg': fig_wrg.to_html(),
              'start_dt': start_dt,
              'end_dt': end_dt,
              'duration': days_hours_minutes(duration),
              'data_sources': md[['Metric', 'Source', 'Frequency']].drop_duplicates().to_html(index=False, classes='data_sources', border=0, justify='left'),
              'descriptive_stats': descriptive_stats.to_html(index=True, border=0, classes='stats_table', justify='left', float_format='{:,.2f}'.format, formatters={'count': int_formatter})}

In [ ]:
with open('./export/index.html', "w", encoding="utf-8") as output_file:
    with open('template.html') as template_file:
        j2_template = Template(template_file.read())
        output_file.write(j2_template.render(jinja_data))